In [1]:
import pandas as pd

In [4]:
path_to_file = '/home/jupyter-d-barkov/Homework/lesson_5/5_data.csv.zip'

In [224]:
retail = pd.read_csv(path_to_file, compression='zip', encoding='ISO-8859-1')

In [225]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [226]:
retail.describe

<bound method NDFrame.describe of        InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

            InvoiceDate  UnitPrice  Custo

In [227]:
retail_columns = retail.columns

In [228]:
retail_columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

### Проверим, встречаются ли в данных повторяющиеся наблюдения. Если они есть, то удалим их из retail.

In [229]:
rows_before_drop = retail.shape[0]

In [230]:
retail.duplicated().value_counts()

False    536641
True       5268
dtype: int64

In [231]:
retail.loc[retail.duplicated()].groupby('InvoiceNo', as_index=False).agg({'StockCode': 'count'}).StockCode.sum()

5268

In [232]:
retail = retail.drop_duplicates()

In [233]:
rows_after_drop = retail.shape[0]
duplicate_rows_num = rows_before_drop - rows_after_drop
duplicate_rows_num

5268

### Посчитаем количество отмененных заказов

In [234]:
canceled_num = len([i for i in retail.InvoiceNo if i.startswith('C')])
canceled_num

9251

In [235]:
canceled_num = retail.InvoiceNo.str.startswith('C').sum()
canceled_num

9251

### Отфильтруем данные, оставив в retail только заказы, где Quantity > 0.

In [236]:
retail = retail.query("Quantity > 0")
retail.shape

(526054, 8)

### Посчитаем число заказов для каждого пользователя (CustomerID) из Германии (Germany).<br> Оставим только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль.<br>Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).

In [237]:
germany_top_df = retail.query("Country == 'Germany'") \
                          .groupby('CustomerID', as_index=False) \
                          .agg({'InvoiceNo': pd.Series.nunique}) \
                          .rename(columns={'InvoiceNo': 'OrderNumber'})

In [238]:
germany_80_procentile = germany_top_df.OrderNumber.quantile(q=0.8)
type(germany_80_procentile)

numpy.float64

In [239]:
germany_top = germany_top_df[germany_top_df.OrderNumber > germany_80_procentile].CustomerID
germany_top

3     12471.0
6     12474.0
8     12476.0
12    12481.0
16    12500.0
26    12524.0
37    12569.0
44    12600.0
51    12619.0
52    12621.0
56    12626.0
64    12647.0
69    12662.0
77    12705.0
78    12708.0
79    12709.0
81    12712.0
83    12720.0
Name: CustomerID, dtype: float64

In [240]:
germany_top = germany_top_df.query('OrderNumber > @germany_80_procentile').CustomerID
germany_top

3     12471.0
6     12474.0
8     12476.0
12    12481.0
16    12500.0
26    12524.0
37    12569.0
44    12600.0
51    12619.0
52    12621.0
56    12626.0
64    12647.0
69    12662.0
77    12705.0
78    12708.0
79    12709.0
81    12712.0
83    12720.0
Name: CustomerID, dtype: float64

###  Отфильтруем наблюдения и оставим в данных записи только по интересующим нас юзерам (germany_top)

In [241]:
top_retail_germany = retail.query("CustomerID in @germany_top")

In [242]:
top_retail_germany

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany
...,...,...,...,...,...,...,...,...
541726,581570,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,12/9/2011 11:59,4.95,12662.0,Germany
541727,581570,23077,DOUGHNUT LIP GLOSS,20,12/9/2011 11:59,1.25,12662.0,Germany
541728,581570,20750,RED RETROSPOT MINI CASES,2,12/9/2011 11:59,7.95,12662.0,Germany
541729,581570,22505,MEMO BOARD COTTAGE DESIGN,4,12/9/2011 11:59,4.95,12662.0,Germany


### Сгруппируем top_retail_germany по коду товара (StockCode).Определим, какой из продуктов добавляли в корзину чаще всего, кроме POST? 

Note: одним заказом считается единовременная покупка любого количества товара, т.е. без учета Quantity.

In [243]:
top_retail_germany.query("StockCode != 'POST'") \
                    .groupby('StockCode', as_index=False) \
                    .agg({'InvoiceNo': 'count'}) \
                    .rename(columns={'InvoiceNo': 'AddNumber'}) \
                    .sort_values('AddNumber', ascending=False)

,StockCode,AddNumber
409,22326,52
411,22328,38
453,22423,34
45,20719,30
709,22961,28
...,...,...
350,22168,1
769,23104,1
768,23102,1
354,22174,1


In [244]:
top_retail_germany.query("StockCode != 'POST'") \
                    .groupby('StockCode') \
                    .agg({'InvoiceNo': 'count'}) \
                    .rename(columns={'InvoiceNo': 'AddNumber'}) \
                    .idxmax()

AddNumber    22326
dtype: object

### Создадим колонку Revenue с суммой покупки в retail, используя колонки Quantity и UnitPrice.

In [245]:
retail = retail.assign(Revenue = retail.Quantity * retail.UnitPrice)

In [246]:
retail.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34


### Для каждой транзакции (InvoiceNo), посчитаем финальную сумму заказа. Выделим топ-5 (InvoiceNo) по сумме заказа.

In [259]:
TotalRevenue_df = retail.groupby('InvoiceNo', as_index=False) \
                        .agg({'Revenue': 'sum'}) \
                        .rename(columns={'Revenue': 'TotalRevenue'})

In [260]:
TotalRevenue_df

,InvoiceNo,TotalRevenue
0,536365,139.12
1,536366,22.20
2,536367,278.73
3,536368,70.05
4,536369,17.85
...,...,...
20723,581586,339.20
20724,581587,249.45
20725,A563185,11062.06
20726,A563186,-11062.06


In [261]:
top_5_TotalRevenue = [int(i) for i in TotalRevenue_df.sort_values('TotalRevenue', ascending=False).head(5).InvoiceNo]

In [262]:
top_5_TotalRevenue

[581483, 541431, 574941, 576365, 556444]

In [270]:
TotalRevenue_df.groupby('InvoiceNo', as_index=False) \
                .agg({'TotalRevenue': 'max'}) \
                .sort_values('TotalRevenue', ascending=False) \
                .head(5)

,InvoiceNo,TotalRevenue
20689,581483,168469.60
2202,541431,77183.60
17582,574941,52940.94
18251,576365,50653.91
9034,556444,38970.00
